# TESTS

In [ ]:
from pathlib import Path
import os
import modflowapi
from modflowapi import Callbacks
from workflow import *
from datetime import datetime

DT_FMT = "%Y-%m-%d %H:%M:%S"


In [ ]:
sim = build_model(ws = 'model', sim_name = 'engesgaard1992', spls = ['sodium'])


In [ ]:
# sim_ws = os.path.join('model', '1dtest', 'chloride')
sim_ws = Path("model/engesgaard1992/")
dll = Path("bin/win/libmf6")

In [ ]:
def try_get_value(address, mf6):
    try:
        res = mf6.get_value(address)
        return res
    except:
        res = 'failed'
        return res

In [ ]:
mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()
# current_time = mf6.get_current_time()
# end_time = mf6.get_end_time()
all_vars = mf6.get_input_var_names() 

# var_add = 

# try:
#     mf6.finalize()
#     success=True
# except:
#     raise RuntimeError

In [ ]:
# current_time, end_time

# all_vars
df_vars = pd.DataFrame(all_vars)[0].str.split('/', expand=True)
df_vars['addr'] = all_vars
df_vars.to_csv('vars.csv')

In [ ]:
val = mf6.get_value(mf6.get_var_address("x", 'SODIUM'))
val += 1

# val = val + 1
val += 1
# mf6.get_value(mf6.get_var_address("x", 'SODIUM')) = val

In [ ]:
mf6.finalize()
val

In [ ]:

mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()

sim_start = datetime.now()
print("...starting transport solution at {0}".format(sim_start.strftime(DT_FMT)))
# reset the node tracking containers

# get the current sim time
ctime = mf6.get_current_time()
ctimes = [0.0]
# get the ending sim time
etime = mf6.get_end_time()
# max number of solution iterations
max_iter = mf6.get_value(mf6.get_var_address("MXITER", "SLN_1"))
num_fails = 0
# let's do it!
while ctime < etime:
    sol_start = datetime.now()
    # length of the current solve time
    dt = mf6.get_time_step()
    # prep the current time step
    mf6.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    mf6.prepare_solve(1)
    # the one-based stress period number
    stress_period = mf6.get_value(mf6.get_var_address("KPER", "TDIS"))[0]
    time_step = mf6.get_value(mf6.get_var_address("KSTP", "TDIS"))[0]

    # solve until converged
    while kiter < max_iter:
        # apply whatever change we want here
        # val = mf6.get_value(mf6.get_var_address("x", 'SODIUM'))
        # val += 1
        convg = mf6.solve(1)
        if convg:
            td = (datetime.now() - sol_start).total_seconds() / 60.0
            print("transport stress period,time step {0},{1} converged with {2} iters, took {3:10.5G} mins".format(stress_period, time_step, kiter,td))
            break
        kiter += 1

    if not convg:
        td = (datetime.now() - sol_start).total_seconds() / 60.0
        print("transport stress period,time step {0},{1} did not converged, {2} iters, took {3:10.5G} mins".format(
            stress_period, time_step, kiter, td))
        num_fails += 1
mf6.finalize()


In [ ]:
def callback_function(sim, callback_step):
    """
    A demonstration function that dynamically adjusts recharge
    and pumping in a modflow-6 model through the MODFLOW-API

    Parameters
    ----------
    sim : modflowapi.ApiSimulation
        A simulation object for the solution group that is
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.get_model()
    if callback_step == Callbacks.initialize:
        print(sim.models)

    # if callback_step == Callbacks.stress_period_start:
    #     # adjust recharge for stress periods 7 through 12
    #     if sim.kper <= 6:
    #         rcha = ml.rcha_0
    #         spd = rcha.stress_period_data
    #         print(f"updating recharge: stress_period={ml.kper}")
    #         spd["recharge"] += 0.40 * sim.kper

    if callback_step == Callbacks.timestep_start:
        print(
            f"Testing: stress_period={ml.kper}, timestep={ml.kstp}", ml.kstp
        )
        if ml.kstp != 2:
            ml.wel.stress_period_data["q"] *= 1/ml.kstp
            # ml.wel.stress_period_data['CONCENTRATION'] = 100000000
            # print('..........................', ml.wel.stress_period_data['CONCENTRATION'])
        else:
            ml.wel.stress_period_data['q'] = 0
        # if ml.kstp ==2:
            
    if callback_step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass
    # return wel

# modflowapi.run_simulation(dll, sim_ws, callback_function, verbose=True)


In [ ]:
ucn = flopy.utils.HeadFile(os.path.join("model",'engesgaard1992',"sodium.ucn"),text="concentration")
carr = ucn.get_alldata()

In [ ]:
# carr[0][0]
carr[-1,0,0,:]